In [ ]:
# Phase 0A: Setup - Install Libraries
%pip install folium geopandas

In [ ]:
# Phase 0B: Setup - Import Libraries
import pandas as pd
import nltk
import re
import requests
import json
import io
import folium
import geopandas as gpd
import os
import time

In [ ]:
# Phase 1A: Load Dataset
# Display 2 sample DataFrame rows
noscemus_metadata = pd.read_csv("https://raw.githubusercontent.com/CCS-ZCU/noscemus_ETF/refs/heads/master/data/metadata_table_long.csv")
noscemus_metadata.head(2)

In [ ]:
# Phase 1B: Inspect DataFrame Structure
# Display DataFrame Columns

print("\nColumns in noscemus_metadata:")
print(noscemus_metadata.columns.tolist())

In [ ]:
# Phase 1C: Examine 'Place' Column
# Inspect Potential Columns
# Replace 'candidate_column_name' with a column name from the list above
candidate_column_name = 'Place' # <-- CHANGE THIS VALUE 

if candidate_column_name in noscemus_metadata.columns:
    print(f"\nUnique values in '{candidate_column_name}':")
    # Display a sample of unique values and their counts
    print(noscemus_metadata[candidate_column_name].value_counts().head(30))
    print(f"\nNumber of unique values in '{candidate_column_name}': {noscemus_metadata[candidate_column_name].nunique()}")
    print(f"Number of missing values in '{candidate_column_name}': {noscemus_metadata[candidate_column_name].isnull().sum()}")
    # Show some raw examples of the data in this column
    print("\nSample raw entries (up to first 20 non-null):")
    print(noscemus_metadata[candidate_column_name].dropna().head(20).tolist())
else:
    print(f"Column '{candidate_column_name}' not found in DataFrame. Please choose from the list printed above.")

In [ ]:
# Phase 2A: Define Place Splitting Logic and Expand Rows

def split_places(place_string):
    if pd.isna(place_string) or not isinstance(place_string, str):
        return [] # Return empty list for NaN or non-string input
    # Split by comma, semicolon, or ampersand. Also handle cases like 'Place1 / Place2'.
    # Regex looks for one or more delimiters, surrounded by optional whitespace.
    places = re.split(r'\s*[,;&/]\s*', place_string)
    # Clean up each individual place name: strip whitespace, remove empty strings
    return [p.strip() for p in places if p and p.strip()] 

expanded_rows = []
if 'noscemus_metadata' in locals():
    print(f"Original number of rows in noscemus_metadata: {len(noscemus_metadata)}")
    for index, row in noscemus_metadata.iterrows():
        original_place_entry = row['Place']
        individual_places = split_places(original_place_entry)
        
        if not individual_places: # Handles NaN, empty strings, or strings that become empty after split
            # Keep the row as is, but ensure 'Place' is None or a consistent empty marker if it was NaN/empty
            new_row = row.copy()
            new_row['Place'] = None # Or np.nan, or an empty string, depending on desired handling for mapping
            expanded_rows.append(new_row)
        elif len(individual_places) == 1:
            # Single place, just copy the row with the cleaned single place name
            new_row = row.copy()
            new_row['Place'] = individual_places[0]
            expanded_rows.append(new_row)
        else:
            # Multiple places, create a new row for each
            for place_name in individual_places:
                new_row = row.copy()
                new_row['Place'] = place_name
                # Add original multi-place string for reference if needed
                new_row['Original_Multi_Place_Entry'] = original_place_entry 
                expanded_rows.append(new_row)
    
    expanded_noscemus_metadata = pd.DataFrame(expanded_rows)
    print(f"Number of rows after expansion: {len(expanded_noscemus_metadata)}")

    # Display a sample, especially focusing on some known multi-place entries to verify
    print("\nSample of expanded_noscemus_metadata (showing some original multi-place entries):")
    # Example: Find rows originating from 'Liegnitz, Wrocław' if it exists
    if 'Original_Multi_Place_Entry' in expanded_noscemus_metadata.columns:
        sample_multi = expanded_noscemus_metadata[expanded_noscemus_metadata['Original_Multi_Place_Entry'] == 'Liegnitz, Wrocław']
        if not sample_multi.empty:
            print(sample_multi[['id', 'Full title', 'Place', 'Original_Multi_Place_Entry']].head())
        else:
            print("Could not find 'Liegnitz, Wrocław' in Original_Multi_Place_Entry for sample.")
        # Show general head as well
        print("\nGeneral head of expanded data:")
        print(expanded_noscemus_metadata[['id', 'Full title', 'Place', 'Original_Multi_Place_Entry' if 'Original_Multi_Place_Entry' in expanded_noscemus_metadata.columns else 'Place']].head())
    else:
        print("\nGeneral head of expanded data (Original_Multi_Place_Entry column not created, likely no multi-place entries found):")
        print(expanded_noscemus_metadata[['id', 'Full title', 'Place']].head())
else:
    print("Error: noscemus_metadata DataFrame not found. Please load it first.")
    expanded_noscemus_metadata = pd.DataFrame() # Initialize empty to avoid errors later


In [ ]:
# Phase 2B: Extract Unique Place Names for Geocoding
# Ensure 'expanded_noscemus_metadata' is available and populated from the previous cell.
if 'expanded_noscemus_metadata' in locals() and not expanded_noscemus_metadata.empty:
    actual_publication_place_column = 'Place' # This is the column with individual place names
    places_series = expanded_noscemus_metadata[actual_publication_place_column].astype(str).str.strip()
    unique_raw_places = places_series.dropna().unique() # Important to dropna here
    print(f"Found {len(unique_raw_places)} unique raw place mentions from '{actual_publication_place_column}' in the expanded data.")
    print("Sample of raw places (first 50 from expanded data):")
    print(unique_raw_places[:50])
else:
    print("Error: expanded_noscemus_metadata is not available or empty. Please ensure the 'Expand Multi-Location Rows' cell ran successfully.")
    # Initialize empty to prevent errors in subsequent cells, or handle appropriately
    places_series = pd.Series(dtype=str) 
    unique_raw_places = []

In [ ]:
# Phase 3A: Geocode Unique Place Names

GEONAMES_USERNAME = "utaysi"  # Your Geonames username
raw_geocoded_cache_file = 'raw_geocoded_places_cache.csv'

def get_coordinates(place_name, username):
    if not place_name or pd.isna(place_name):
        return None, None, None, None
    # Ensure place_name is a string for requests.utils.quote
    place_name_str = str(place_name)
    try:
        # Initial attempt: prioritize populated places (featureClass=P)
        url = f"http://api.geonames.org/searchJSON?q={requests.utils.quote(place_name_str)}&maxRows=1&featureClass=P&username={username}"
        response = requests.get(url, timeout=15)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        if data.get('geonames') and len(data['geonames']) > 0:
            top_result = data['geonames'][0]
            return float(top_result['lat']), float(top_result['lng']), top_result.get('name'), top_result.get('countryName')
        else:
            # Fallback: search without featureClass if no populated place found or if initial result is empty
            # This helps with broader terms or historical names that might not be classed as 'P'
            url_fallback = f"http://api.geonames.org/searchJSON?q={requests.utils.quote(place_name_str)}&maxRows=1&username={username}"
            # print(f"Retrying without featureClass for: {place_name_str}") # Optional: for debugging
            response_fallback = requests.get(url_fallback, timeout=15)
            response_fallback.raise_for_status()
            data_fallback = response_fallback.json()
            if data_fallback.get('geonames') and len(data_fallback['geonames']) > 0:
                top_result_fallback = data_fallback['geonames'][0]
                # print(f"Fallback success for {place_name_str}: Found {top_result_fallback.get('name')}") # Optional
                return float(top_result_fallback['lat']), float(top_result_fallback['lng']), top_result_fallback.get('name'), top_result_fallback.get('countryName')
            # print(f"Place not found by Geonames (even after fallback): {place_name_str}") # Optional
            return None, None, None, None
    except requests.exceptions.Timeout:
        print(f"API request timed out for {place_name_str}")
        return None, None, None, None
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred for {place_name_str}: {http_err} - Response: {response.text[:200]}...")
        return None, None, None, None
    except requests.exceptions.RequestException as req_err:
        print(f"API request failed for {place_name_str}: {req_err}")
        return None, None, None, None
    except ValueError as json_err: # Handles JSON decoding errors
        print(f"JSON decoding failed for {place_name_str} (response: {response.text[:200]}...): {json_err}")
        return None, None, None, None

# Check for cached data first
if os.path.exists(raw_geocoded_cache_file):
    print(f"Loading raw geocoded data from cache: {raw_geocoded_cache_file}")
    raw_geocoded_df = pd.read_csv(raw_geocoded_cache_file)
    # Ensure all expected columns are present, fill with NA if not
    expected_cols = ['raw_place', 'geoname_name', 'latitude', 'longitude', 'country']
    for col in expected_cols:
        if col not in raw_geocoded_df.columns:
            raw_geocoded_df[col] = pd.NA
else:
    print(f"No cache file found ({raw_geocoded_cache_file}). Geocoding raw places...")
    raw_geocoded_data = []
    if 'places_series' in locals():
        unique_raw_places = places_series.dropna().unique() # Use dropna() before unique()
        print(f"Geocoding {len(unique_raw_places)} unique raw place names...")
        for i, place in enumerate(unique_raw_places):
            if str(place).strip() == "nan" or str(place).strip() == "": # Skip if place is 'nan' string or empty after strip
                # print(f"Skipping invalid place entry: '{place}'") # Optional
                lat, lon, geoname_name, country = None, None, None, None
            else:
                if (i+1) % 20 == 0:
                    print(f"Processed {i+1}/{len(unique_raw_places)} places...")
                lat, lon, geoname_name, country = get_coordinates(place, GEONAMES_USERNAME)
            
            raw_geocoded_data.append({'raw_place': place, 
                                      'geoname_name': geoname_name, 
                                      'latitude': lat, 
                                      'longitude': lon, 
                                      'country': country})
            time.sleep(0.1) # 100ms delay to be respectful to the API

        raw_geocoded_df = pd.DataFrame(raw_geocoded_data)
        raw_geocoded_df.to_csv(raw_geocoded_cache_file, index=False)
        print(f"Saved raw geocoded data to cache: {raw_geocoded_cache_file}")
    else:
        print("Error: 'places_series' not defined. Please ensure the previous cells (especially 'cline_extract_place_column') have been run.")
        raw_geocoded_df = pd.DataFrame(columns=['raw_place', 'geoname_name', 'latitude', 'longitude', 'country']) # Create empty df

if not raw_geocoded_df.empty:
    print(f"\nSuccessfully geocoded {raw_geocoded_df['latitude'].notna().sum()} places out of {len(raw_geocoded_df)} unique raw names processed.")
    print("\nSample of geocoded data (first 20 rows):")
    print(raw_geocoded_df.head(20))
    
    print("\nPlaces that were NOT found by Geonames (sample):")
    not_found_sample = raw_geocoded_df[raw_geocoded_df['latitude'].isna()]['raw_place'].unique()
    print(not_found_sample[:20]) # Show up to 20 unique not found raw places
    print(f"Total unique raw places not found: {len(not_found_sample)}")
else:
    print("\nraw_geocoded_df is empty. Check for errors in previous steps or API calls.")